# Resonant inelastic X-ray scattering (RIXS) 

In this exercise, you will analyze a RIXS measurement for Ni MOF. RIXS, also known as resonant X-ray emission spectroscopy (RXES), is a photon-in/photon-out experiment. The incident energy Ω is scanned across an absorption edge. The emitted energy ω is also scanned either over the fluorescence lines or over energies just below the elastically scattered peak. In the latter case, the energy transfer Ω-ω becomes small (on the order of a few eV), and valence band excitations are observed.

<figure>
  <img src="assets/rixs.png" alt="XAS" style="width:40%">
  <figcaption style="text-align: center; font-style: italic">P. Glatzel et al., The European Physical Journal Special Topics 169, 207 (2009).
</figcaption>
</figure>

The RIXS plane (bottom left) is based on the above energy diagram and corresponding line plots. The lifetime broadenings for the intermediate state and final state are shown as straight lines in the plane. Line plots are shown at constant incident energy (bottom right) and constant final state energy or constant energy transfer (top left). A diagonal line through the RIXS plane (dashed) corresponds to a scan of the incident energy at fixed emission energy (HERFD). 

<figure>
  <img src="assets/rixs_plane.png" alt="XAS" style="width:60%">
  <figcaption style="text-align: center; font-style: italic">P. Glatzel et al., The European Physical Journal Special Topics 169, 207 (2009).
</figcaption>
</figure>

# Import the required packages

In [ ]:
%matplotlib ipympl

import sys
import logging

import numpy as np
import scipy
import matplotlib.pyplot as plt
import silx.io.h5py_utils

## Load the experimental data

Below you will find the filename and the counters/positioners for the RIXS measurement. As before, the file is already stored on the local disk, but you can also download it from the ESRF data portal (https://data.esrf.fr/public/10.15151/ESRF-DC-924121042, `Sami_Ni_MOF_RIXS_cc`, `Ka1`). Contrary to what you did until now, you will need to construct the RIXS plane by stacking the individual scans; no more averaging. You need to include the scans 3 to 83.

Start by creating a list with the scan indices.

In [ ]:
filename = "experimental_data/ihch1527/id26/Sami_Ni_MOF_RIXS_cc/Sami_Ni_MOF_RIXS_cc_Ka1/Sami_Ni_MOF_RIXS_cc_Ka1.h5"

data_mappings = {
    "x": ".1/measurement/hdh_energy",
    "y": ".1/instrument/positioners/xes_en",
    "signal": ".1/measurement/det_dtc_apd", 
    "monitor": ".1/measurement/I02",
}

# Create a list with the scan indices.
##################
# YOUR CODE HERE # 
scans_ids = list(range(3, 84))
##################

Loop over the scans and add the data to lists.

In [ ]:
x, y, signal, monitor = [], [], [], []

with silx.io.h5py_utils.File(filename) as sf:
    for scan_id in scans_ids:
        x_scan = sf[f"{scan_id}{data_mappings['x']}"][()]
        signal_scan = sf[f"{scan_id}{data_mappings['signal']}"][()]
        monitor_scan = sf[f"{scan_id}{data_mappings['monitor']}"][()]
        
        x.extend(x_scan)
        signal.extend(signal_scan)
        monitor.extend(monitor_scan)
        
        # Make the y array by repeating the value n times, where n
        # is the size of the x array.
        ##################
        # YOUR CODE HERE #         
        y_value = sf[f"{scan_id}{data_mappings['y']}"][()]
        y.extend(y_value * np.ones_like(x_scan))        
        ##################

Convert the lists to Numpy arrays.

In [ ]:
x = np.array(x)
y = np.array(y)
signal = np.array(signal)
monitor = np.array(monitor)

## A note on plotting 2D data

Matplotlib offers two functions to plot 2D data. Change the function parameters to assess their effects on the plot.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7.8, 4.5), sharey=True)

# Plot using tricontourf.
levels = 11
ax1.tricontourf(x, y, signal, levels=levels)
ax1.set_xlabel("Incident energy (keV)")
ax1.set_ylabel("Emission energy (keV)")
ax1.set_title("Using tricontourf")

# Plot using imshow. The data needs to be reshaped.
nscans = 81
npoints_per_scan = 66
signal_reshaped = signal.reshape((nscans, npoints_per_scan))

# https://matplotlib.org/stable/gallery/images_contours_and_fields/interpolation_methods.html
interpolation = "bilinear"
extent = [x.min(), x.max(), y.min(), y.max()]
ax2.imshow(
    signal_reshaped,
    origin="lower",
    extent=extent,
    aspect="auto",
    interpolation=interpolation,
)
ax2.set_xlabel("Incident energy (keV)")
ax2.set_title("Using imshow")
plt.tight_layout()

## Emission energy vs energy transfer plots

RIXS planes are usually plotted using energy transfer on the Y-axis, which is defined as the difference between the incident and the emission energies. Plot the two repesentations.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4.5))

# Plot the plane with the emission energy as Y-axis.
ax1.tricontourf(x, y, signal, levels=levels)
ax1.set_xlabel("Incident energy (keV)")
ax1.set_ylabel("Emission energy (keV)")

# Plot the plane with the energy transfer as the Y-axis.
##################
# YOUR CODE HERE # 
ax2.tricontourf(x, x - y, signal, levels=levels)
##################
ax2.set_xlabel("Incident energy (keV)")
ax2.set_ylabel("Energy transfer (keV)")

plt.tight_layout()

## Cut the RIXS plane

To make things easier, you will use the `daxs` library to perform the cuts. Discuss a method that you can use to perform arbitrary cuts in a 2D image.

In [ ]:
from daxs.measurements import Hdf5Source, Rixs

source = Hdf5Source(filename=filename, included_scans=scans_ids, data_mappings=data_mappings)
measurement = Rixs(source)

In [ ]:
measurement.cut("CEE", energies=[7.4780, 7.4790, 7.4794])
measurement.cut("CIE", energies=[8.332])
measurement.cut("CET", energies=[0.8531])

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8.5, 4.5))

ax1.tricontourf(measurement.x, measurement.y, measurement.signal, levels=levels)

for label, (x, y, signal) in measurement.cuts.items():
    if "CIE" in label:
        continue
        
    ax1.plot(x, y)

    ids = np.argsort(x)
    x = x[ids]
    y = y[ids]
    signal = signal[ids]

    ax2.plot(x, signal, label=label)

ax1.set_xlabel("Incident energy (keV)")
ax1.set_ylabel("Energy transfer (keV)")

ax2.set_xlabel("Incident energy (keV)")
ax2.set_ylabel("Intensity (arb. units)")

ax2.legend() 
plt.tight_layout()